In [169]:
from IPython.display import HTML

import jinja2 as jj
import numpy as np
import pandas as pd

In [170]:
def get_dataframe(name):
    data_url = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
               f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    
    return pd.read_csv(data_url)

In [171]:
COL_COUNTRY = 'Country/Region'
COL_STATE = 'Province/State'
COUNTRY = 'Australia'

dft_confirm = get_dataframe('confirmed')
dft_confirm = dft_confirm[dft_confirm[COL_COUNTRY] == COUNTRY]
 
dft_demised = get_dataframe('deaths')
dft_demised = dft_demised[dft_demised[COL_COUNTRY] == COUNTRY]

dft_recover = get_dataframe('recovered')
dft_recover = dft_recover[dft_recover[COL_COUNTRY] == COUNTRY]

In [172]:
COL_TODAY = dft_confirm.columns[-1]
COL_1DAY = dft_confirm.columns[-1 - 1]
COL_5DAY = dft_confirm.columns[-1 - 5]
COL_50DAY = dft_confirm.columns[-1 - 50]

df_table = pd.DataFrame({'State': dft_confirm[COL_STATE], 'Cases': dft_confirm[COL_TODAY], 
                         'Recover': dft_recover[COL_TODAY], 'Deaths': dft_demised[COL_TODAY]})

df_table['Cases (5D)'] = np.array(dft_confirm[COL_TODAY]) - np.array(dft_confirm[COL_5DAY])
df_table['Recover (5D)'] = np.array(dft_recover[COL_TODAY]) - np.array(dft_recover[COL_5DAY])
df_table['Deaths (5D)'] = np.array(dft_demised[COL_TODAY]) - np.array(dft_demised[COL_5DAY])
df_table['Cases (1D)'] = np.array(dft_confirm[COL_TODAY]) - np.array(dft_confirm[COL_1DAY])
df_table['Recover (1D)'] = np.array(dft_recover[COL_TODAY]) - np.array(dft_recover[COL_1DAY])
df_table['Deaths (1D)'] = np.array(dft_demised[COL_TODAY]) - np.array(dft_demised[COL_1DAY])
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)

df_table = df_table.sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
df_table = df_table.reset_index()
df_table.index += 1

del df_table['index'] # del duplicate index
df_table.head(8)

,State,Cases,Recover,Deaths,Cases (5D),Recover (5D),Deaths (5D),Cases (1D),Recover (1D),Deaths (1D),Fatality Rate
1,New South Wales,1617,4,8,948,0,2,212,0,1,0.5
2,Victoria,685,191,3,330,121,3,111,20,0,0.4
3,Queensland,625,8,1,306,0,1,70,0,0,0.2
4,South Australia,287,6,0,153,3,0,30,0,0,0.0
5,Western Australia,278,28,2,138,28,1,47,27,0,0.7
6,Australian Capital Territory,71,2,0,39,2,0,9,1,0,0.0
7,Tasmania,62,5,0,34,2,0,15,2,0,0.0
8,Northern Territory,15,0,0,10,0,0,3,0,0,0.0


In [173]:
dt_cols = dft_confirm.columns[~dft_confirm.columns.isin([COL_STATE, COL_COUNTRY, 'Lat', 'Long'])]
dft_cases = dft_confirm.groupby(COL_STATE)[dt_cols].sum()
dft_cases_new = dft_cases.diff(axis=1).fillna(0).astype(int)

include_cols = ['Cases', 'Recover', 'Deaths', 'Cases (5D)', 'Recover (5D)', 'Deaths (5D)']

summary_nsw = df_table[df_table['State'].eq('New South Wales')][include_cols].sum().add_prefix('NSW ')
summary_vic = df_table[df_table['State'].eq('Victoria')][include_cols].sum().add_prefix('VIC ')
summary_qld = df_table[df_table['State'].eq('Queensland')][include_cols].sum().add_prefix('QLD ')

summary_time = {'updated': pd.to_datetime(COL_TODAY), 'since': pd.to_datetime(COL_5DAY)}
summary = {**summary_time, **df_table[include_cols].sum(), **summary_nsw, **summary_vic, **summary_qld}
summary

{'updated': Timestamp('2020-03-28 00:00:00'),
 'since': Timestamp('2020-03-23 00:00:00'),
 'Cases': 3640,
 'Recover': 244,
 'Deaths': 14,
 'Cases (5D)': 1958,
 'Recover (5D)': 156,
 'Deaths (5D)': 7,
 'NSW Cases': 1617,
 'NSW Recover': 4,
 'NSW Deaths': 8,
 'NSW Cases (5D)': 948,
 'NSW Recover (5D)': 0,
 'NSW Deaths (5D)': 2,
 'VIC Cases': 685,
 'VIC Recover': 191,
 'VIC Deaths': 3,
 'VIC Cases (5D)': 330,
 'VIC Recover (5D)': 121,
 'VIC Deaths (5D)': 3,
 'QLD Cases': 625,
 'QLD Recover': 8,
 'QLD Deaths': 1,
 'QLD Cases (5D)': 306,
 'QLD Recover (5D)': 0,
 'QLD Deaths (5D)': 1}

In [174]:
templateLoader = jj.FileSystemLoader(searchpath='./')
templateEnv = jj.Environment(loader=templateLoader)

template = templateEnv.get_template('templates/overview.html')
html_text = template.render(D=summary, table=df_table, newcases=dft_cases_new, np=np, pd=pd, enumerate=enumerate)

with open('.local/temp1.html', 'w') as f:
    f.write(html_text)

HTML(f'<div>{html_text}</div>')
